# Schema: schema's in MongoDB


## Validator: (partieel) schema voor collection-documenten

Elk document in een MongoDB collection heeft zijn eigen *structuur*: veldnamen en bijbehorende waarden (types).
Deze grote vrijheid is niet handig als je een collectie wilt kunnen doorzoeken:
daarvoor moet je weten welke namen en waarden de verschillende documenten gebruiken.
Dit werkt beter als die documenten een bepaalde (minimale) gemeenschappelijke structuur hebben.

Met behulp van een *validator* kun je een *minimale* structuur van de documenten in een collection beschrijven.
MongoDB gebruikt deze validator bij het toevoegen of aanpassen van een document.
Als dit document niet voldoet aan de beschrijving van de validator, wordt het niet toegevoegd.

Je kunt de validator opgeven bij het aanmaken van de collection.
Je kunt deze ook later toevoegen, met het db-commando `collMod` voor het aanpassen van de eigenschappen van een collection.

### Schema

De structuur van de documenten in een collection noemen we een *schema*.
In een MongoDB collection-schema bepaal je zelf welk deel van de structuur vastligt, 
en waar documenten kunnen verschillen.

> In een SQL database beschrijft het (fysieke) schema de *volledige structuur* van de database: de tabellen, en de structuur van elke tabel (namen en types van de kolommen).
Alle rijen (records) in een tabel hebben dezelfde structuur.


## Initialisaties

In [ ]:
import os
import re
import pandas as pd
import numpy as np
from IPython.core.display import display, HTML
import pymongo

pd.set_option('max_colwidth',160)

userline = !echo $USER
username = userline[0]
dbname = username + "-demodb"
print("Database name: " + dbname)

from pymongo import MongoClient
print('Mongo version', pymongo.__version__)
client = MongoClient('localhost', 27017)
db = client[dbname]


mongopathfile = !cat mongopath
mongopath = mongopathfile[0]

contacts = db.contacts
contacts.drop()
os.system(mongopath + 'mongoimport -d ' + dbname + ' -c contacts adressen.json')

## Tegenvoorbeeld: toevoegen van een vreemd document

MongoDB-collections hebben in eerste instantie geen structuur (schema).
Dit betekent dat je willekeurige documenten kun toevoegen, zoals we hier demonstreren:

In [ ]:
contacts.insert_one({"kleur": "groen", "prijs": 400, "beschrijving": "fiets"})

In [ ]:
list(contacts.find())

Dit is natuurlijk niet de bedoeling.
Als de database sterk gekoppeld is aan een enkele toepassing zal dit niet zo snel gebeuren.
Maar een database wordt vaak door meerdere toepassingen gebruikt: je wilt dergelijke problemen dan voorkomen.

Bovendien wil je weten welke velden (properties) gebruikt kunnen worden in de documenten in een bepaalde collection, zoals `contacts` of `agenda`.

## Toevoegen van de validator

Met behulp van een *validator* controleert MongoDB bij het toevoegen of aanpassen van een document in een collection of dat document voldoet aan de regels van die collection.

Je kunt een validator zien als een query-expressie waarmee je alle "valide" documenten in de database filtert.

We kunnen de validator van een collection instellen met behulp van het db-commando `collMod`.

### Definiëren van de validator

Als minimale eis voor de documenten in de `contacts`-collection stellen we dat er tenminste een `name`-veld (property) is, en een `email` of een `tel`-veld.
Dit beschrijven we met het volgende schema: 

In [ ]:
contact_schema = {"name": {"$type": "string"},
                  "$or": [{"email": {"$type": "string"}},
                          {"tel": {"$type": "string"}}

                         ]}

We testen dit schema, door te zoeken naar de documenten die hier wel aan voldoen:

In [ ]:
list(contacts.find(contact_schema))

### Toevoegen van de validator aan de collection

We voegen dit schema toe als *validator*-schema voor de collection `contacts`.

> Je kunt de validator definiëren bij de initialisatie van de collection, maar je kunt deze achteraf ook nog veranderen, zoals we hier doen.

In [ ]:
db.command("collMod", "contacts", validator=contact_schema)

### Voorbeeld: toevoegen van een valide document

Het toevoegen van een document dat aan deze regels voldoet:

In [ ]:
contacts.insert_one({"name": "Henk de Vries", "tel": "06 3333 8765"})

### Voorbeeld: toevoegen van een niet-valide document

Het toevoegen van een document dat *niet* aan deze regels voldoet (door een foute keuze van het "name"-veld).

> Dit geeft een foutmelding; later geven we een manier om hier handiger mee om te gaan in een programma.

In [ ]:
contacts.insert_one({"naam": "Anne de Boer", "tel": "06 1234 8855"})

Het is handiger om dergelijke fouten in het programma zelf op te vangen.
Python biedt hiervoor de mogelijkheid met het exception-mechanisme, zie het voorbeeld hieronder:

In [ ]:
try:
    contacts.insert_one({"naam": "Anne de Boer", "tel": "06 1234 8855"})
except pymongo.errors.WriteError as s:
    print("Document not inserted: " + str(s))
else:
    print("insert OK")

## Vinden van niet-valide documenten

Als je achteraf de validator aanpast, kan de database documenten bevatten die niet aan deze nieuwe validator voldoen.
Met behulp van de volgende `"$nor"`-constructie kun je deze niet-valide documenten vinden.
(Een `nor` met 1 alternatief is gelijk aan een `not` van dat alternatief.)

In [ ]:
list(contacts.find({"$nor": [contact_schema]}))

## Opdracht

* breid het schema uit zodat naast een document naast de naam, een telefoonnummer of een emailadres of een fysiek adres  bevat. Dit fysieke adres heeft (tenminste) de eigenschappen (properties) `street` en `city`.
  (Herdefinieer `contact_schema` voor deze voorwaarden.)

* zoek alle documenten die aan dit schema voldoen.

* (her)definieer de collection-validator met dit nieuwe schema.

* zoek alle documenten die *niet* aan dit schema voldoen.

Demonstreer dat het schema goed werkt bij het toevoegen voor het volgende document.

> Ga zelf na of dit aan het schema voldoet. Welk resultaat verwacht je?

In [ ]:
person = {"name": "Henk de Vries",
          "address": {"street": "Kastanjelaan 31", "city": "Almere"}}
try:
    contacts.insert_one(person)
except pymongo.errors.WriteError as s:
    print("Document not inserted: " + str(s))
else:
    print("insert OK")

## Opdracht

We willen alleen adressen met (tenminste) `street`, `city` en `postcode` toestaan.

* Herdefinieer het `address_schema` zodat ook de postcode hierin opgenomen is als string.

> *Opmerking*: je kunt met behulp van reguliere expressies nog preciezer beschrijven hoe een postcode eruit kan zien,
maar dat later we hier buiten beschouwing. `string` is voldoende.
Herdefinieer de validator met dit nieuwe `address_schema`.

* Geef een voorbeeld van een insert van een document dat voldoet aan deze validator.

* Geef een voorbeeld van een insert van een document dat *niet* voldoet aan deze validator. 

## Opmerkingen

* voor het beschrijven van een schema voor een validator biedt MongoDB twee mogelijkheden:
    * de oorspronkelijke MongoDB-query-notatie, zoals hierboven gebruikt;
    * JSON-schema, een (internet/IETF) draft-standaard voor JSON-documenten (zie https://json-schema.org, en https://json-schema.org/latest/json-schema-core.html).